In [4]:
import json
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib.colors import Normalize
import os
from tqdm import tqdm

# 1. 출력 디렉토리 설정
output_dir = r'C:\baramproject\wind_visualize'
os.makedirs(output_dir, exist_ok=True)  # 디렉토리 없으면 생성

# 2. 2018년 1월 1일 00:00부터 23:30까지 시간 목록 (30분 단위)
times = [f'{h:02d}{m:02d}' for h in range(24) for m in [0, 30]]  # 0000, 0030, ..., 2330

# 3. 각 시간별 JSON 파일 시각화 및 저장
for time in tqdm(times, desc="시각화 진행률"):
    # JSON 파일 경로
    json_file = rf'C:\baramproject\wind_database\wind_data_20180101_{time}.json'
    
    # 파일 존재 확인
    if not os.path.exists(json_file):
        print(f"파일이 존재하지 않습니다: {json_file}")
        continue

    # 4. JSON 파일 로드
    with open(json_file, 'r') as f:
        data = json.load(f)

    # 5. 데이터 추출
    metadata = data['metadata']
    lats = [item['lat'] for item in data['data']]
    lons = [item['lon'] for item in data['data']]
    speeds = [item['speed'] for item in data['data']]
    directions = [item['direction'] for item in data['data']]

    # 6. 그리드 크기 및 좌표 정리 (33x29로 정렬)
    grid_size = metadata['grid_size']
    n_rows, n_cols = grid_size['rows'], grid_size['cols']  # 33, 29

    # 위도와 경도를 2D 배열로 변환 (0.25도 간격으로 정렬 가정)
    lat_grid = np.linspace(metadata['region']['lat_max'], metadata['region']['lat_min'], n_rows)  # 38.0 ~ 30.0
    lon_grid = np.linspace(metadata['region']['lon_min'], metadata['region']['lon_max'], n_cols)  # 120.0 ~ 127.0
    lon_2d, lat_2d = np.meshgrid(lon_grid, lat_grid)

    # U-풍과 V-풍으로 변환 (풍향을 북쪽 기준으로 U, V 성분 계산, 크기 고정)
    u = [np.sin(np.radians(dir)) for dir in directions]  # 동-서 성분 (크기 1로 고정)
    v = [np.cos(np.radians(dir)) for dir in directions]  # 북-남 성분 (크기 1로 고정)

    # U, V 데이터를 2D 배열로 재구성
    u_2d = np.reshape(u, (n_rows, n_cols))
    v_2d = np.reshape(v, (n_rows, n_cols))
    speed_2d = np.reshape(speeds, (n_rows, n_cols))

    # 7. 지도 시각화
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_extent([120, 127, 30, 38], crs=ccrs.PlateCarree())  # 지정된 범위

    # 지도 특징 추가
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='white')
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')

    # 화살표 플롯 (풍향은 방향으로만, 풍속은 색상으로 표시)
    norm = Normalize(vmin=np.min(speed_2d), vmax=np.max(speed_2d))
    quiver = ax.quiver(lon_2d, lat_2d, u_2d, v_2d, speed_2d, cmap='coolwarm', norm=norm, scale=50, width=0.003, headwidth=4, headlength=6)

    # 색상 바 추가 (풍속)
    cbar = plt.colorbar(quiver, ax=ax, orientation='vertical', pad=0.05, aspect=50)
    cbar.set_label('Wind Speed (m/s)')

    # 제목 추가
    plt.title(f'Wind Direction (Arrows) and Speed (Color) - {metadata["timestamp"]}')

    # 격자선 추가 (선택 사항)
    ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5)

    # 8. 시각화 결과 저장
    output_file = os.path.join(output_dir, f'wind_visualization_20180101_{time}.png')
    plt.savefig(output_file, dpi=300, bbox_inches='tight')
    plt.close(fig)  # 메모리 관리

    print(f"저장 완료: {output_file}")

print("모든 시각화 이미지가 성공적으로 저장되었습니다!")

시각화 진행률:   2%|█▍                                                                  | 1/48 [00:00<00:22,  2.10it/s]

저장 완료: C:\baramproject\wind_visualize\wind_visualization_20180101_0000.png


시각화 진행률:   4%|██▊                                                                 | 2/48 [00:01<00:23,  1.96it/s]

저장 완료: C:\baramproject\wind_visualize\wind_visualization_20180101_0030.png


시각화 진행률:   6%|████▎                                                               | 3/48 [00:01<00:22,  2.03it/s]

...

저장 완료: C:\baramproject\wind_visualize\wind_visualization_20180101_2230.png


시각화 진행률:  98%|█████████████████████████████████████████████████████████████████▌ | 47/48 [00:22<00:00,  2.12it/s]

저장 완료: C:\baramproject\wind_visualize\wind_visualization_20180101_2300.png


시각화 진행률: 100%|███████████████████████████████████████████████████████████████████| 48/48 [00:22<00:00,  2.09it/s]

저장 완료: C:\baramproject\wind_visualize\wind_visualization_20180101_2330.png
모든 시각화 이미지가 성공적으로 저장되었습니다!
